In [0]:
#Importing required packages
import tensorflow as tf
from keras.applications import inception_v3
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import BatchNormalization, Dense, RepeatVector
import os, glob
import json, pickle
import numpy as np
import tensorflow_hub as hub


Using TensorFlow backend.
W0502 23:58:59.495810 140170155288448 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
#Loading the inception_v3 model
#We are not considering the top fully connected layer and just taking the 2048 vector from convolutions
module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")

Instructions for updating:
Colocations handled automatically by placer.


W0502 23:59:04.644039 140170155288448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Code to connect the google drive to colab
"""

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

"""

In [0]:
#Seeing the contents of the drive
"""
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive
"""

In [0]:
#unzipping the required image data set
!unzip drive/DL/DL_Project/Flickr8k_Dataset.zip

In [0]:
#Getting all the image files in the prescribed path
path = "Flicker8k_Dataset/***.jpg"
file_path = glob.glob(path)
print (len(file_path))
#    for file in file_path:
print (file_path[0])

8091
Flicker8k_Dataset/2934837034_a8ca5b1f50.jpg


In [0]:
#Creating a place holder, this will be used to pass image to inception model to get 2048 vector
X = tf.placeholder(tf.float32, shape = (None, 299,299,3))
#Getting the output of the inception model
out = module(X)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0503 00:09:42.753012 140170155288448 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
#setting up the tensor flow session
session = tf.InteractiveSession()
tf.global_variables_initializer().run()

#creating list to store image names and image vectors
input_images = []
image_names = []
#Getting each image and processing it
for i in range(len(file_path)):
#  print (i)
#Adding image name to image names list
  image_names.append(file_path[i])
#loading image in required format (image shape required for inception network is 299,299,3)
  img = load_img(file_path[i], target_size=(299, 299))
  img_array = img_to_array(img)
#Processing the image data, normalizing the oixel data from -1 to 1
  img_array = inception_v3.preprocess_input(img_array)
#Reshaping the image data to pass to inception network
  img_array = img_array.reshape(1,img_array.shape[0], img_array.shape[1], img_array.shape[2])
#passing image data to inception network and getting 2048 vector
  out_array = session.run([out], feed_dict={X:img_array})
#storing the 2048 vectors in a input_images list
  input_images.append(np.array(out_array))


print (type(input_images))
input_images = np.array(input_images)
#input_images = input_images.rehape(10,2048)
print (type(input_images))
print (input_images.shape)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


<class 'list'>
<class 'numpy.ndarray'>
(8091, 1, 1, 2048)


In [0]:
#rehaping the input list from 8091,1,1,2048 to 8091,2048 shape
input_images = input_images.reshape(input_images.shape[0],2048)
print (type(input_images))
print (input_images.shape)


<class 'numpy.ndarray'>
(8091, 2048)


In [0]:
#Converting the names list to numpy array
print (type(image_names))
print (len(image_names))
image_names = np.array(image_names)
print (image_names.shape)

<class 'list'>
8091
(8091,)


In [0]:
print (input_images[0])
print (image_names[0])

[0.36827826 0.17485881 0.00189675 ... 0.16599445 0.13291389 0.10068121]
Flicker8k_Dataset/2934837034_a8ca5b1f50.jpg


In [0]:
#Storing both the image names array and image vector array in the drive, this was done to readily use this information
with open('drive/DL/DL_Project/inception_2048_vector.pkl','wb') as f:
  pickle.dump(input_images, f)
  
with open('drive/DL/DL_Project/images_file_names_vector_index.pkl','wb') as f:
  pickle.dump(image_names, f)